In [1]:
import glob
import pandas as pd
import os
import librosa
import random

import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl

from BEATs.Tokenizers import TokenizersConfig, Tokenizers
from BEATs.BEATs import BEATs, BEATsConfig
from transferLearning import BEATsTransferLearningModel
from ECS50DataModule import ECS50DataModule

/home/said/anaconda3/envs/openmmlab/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/said/anaconda3/envs/openmmlab/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")
/home/said/anaconda3/envs/openmmlab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_folder = "/home/said/projects/BirdCLEF/Fine-tune-BEATs/ESC-50-master"
audio = glob.glob(data_folder + "/audio/*.wav", recursive=True)
labels = pd.read_csv(os.path.join(data_folder, "meta/esc50.csv"))

In [3]:
labels.head()

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


In [4]:
df_audio = pd.DataFrame(audio, columns=["filepath"])
df_audio["filename"] = [f.split("/")[-1] for f in audio]
filepath_labels = labels.merge(df_audio, how="inner", on="filename")

In [5]:
filepath_labels.head()

,filename,fold,target,category,esc10,src_file,take,filepath
0,1-100032-A-0.wav,1,0,dog,True,100032,A,/home/said/projects/BirdCLEF/Fine-tune-BEATs/E...
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A,/home/said/projects/BirdCLEF/Fine-tune-BEATs/E...
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A,/home/said/projects/BirdCLEF/Fine-tune-BEATs/E...
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B,/home/said/projects/BirdCLEF/Fine-tune-BEATs/E...
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A,/home/said/projects/BirdCLEF/Fine-tune-BEATs/E...


In [6]:
# load the pre-trained checkpoints
# checkpoint = torch.load('/home/said/projects/BirdCLEF/Fine-tune-BEATs/checkpoints/BEATs_iter3_plus_AS2M.pt')
# cfg = BEATsConfig(
#             {
#                 **checkpoint["cfg"]
#             }
#         )
# BEATs_model = BEATs(cfg)

# ft = torch.load("/app/lightning_logs/version_0/checkpoints/epoch=9-step=500.ckpt")
# ft["state_dict"] = {key.replace('beats.', ''): value 
#                     for key, value in ft["state_dict"].items() if not key.endswith(('fc.weight', 'fc.bias'))}
# BEATs_model.load_state_dict(ft["state_dict"])

# Fine Tune BEATs

In [8]:
filepath_labels.target.unique().shape

(50,)

In [15]:
model = BEATsTransferLearningModel(
    num_target_classes=50,
    milestones=5,
    batch_size=1,
    lr=1e-3,
    lr_scheduler_gamma=1e-1,
    num_workers=2,
    model_path="/home/said/projects/BirdCLEF/Fine-tune-BEATs/checkpoints/BEATs_iter3_plus_AS2M.pt"
)


In [17]:
# Create an instance of the ECS50DataModule
data_module = ECS50DataModule(root_dir = "/home/said/projects/BirdCLEF/Fine-tune-BEATs/ESC-50-master/audio/",
                            csv_file = "/home/said/projects/BirdCLEF/Fine-tune-BEATs/ESC-50-master/meta/esc50.csv",
                            batch_size=1,
                            split_ratio=0.8,
                            transform=None)

In [18]:
train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()

In [19]:
trainer = pl.Trainer(
    max_epochs=1,
    #gpus=1,
    num_nodes=1,
    #checkpoint_callback=True,
    #progress_bar_refresh_rate=30,
    logger=None
)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [20]:
trainer.fit(model, train_loader, val_loader)


  | Name      | Type               | Params
-------------------------------------------------
0 | beats     | BEATs              | 90.3 M
1 | fc        | Linear             | 38.5 K
2 | train_acc | MulticlassAccuracy | 0     
3 | valid_acc | MulticlassAccuracy | 0     
-------------------------------------------------
90.4 M    Trainable params
0         Non-trainable params
90.4 M    Total params
361.401   Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/said/anaconda3/envs/openmmlab/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/said/anaconda3/envs/openmmlab/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   1%|          | 14/1600 [00:34<1:04:14,  2.43s/it, v_num=2, train_acc=0.000]

/home/said/anaconda3/envs/openmmlab/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
